In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import time
import timeit
import datetime
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import http.client
import requests
import json
import yfinance as yf
#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from heapq import *
import planarity
from sklearn.linear_model import LinearRegression
import scipy as sc
import math
import random

#!pip install finta
from finta import TA
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

##### Data downloaded on Nov.29

In [ ]:
whole_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

##### Obtain current active components of S&P500

In [ ]:
# df_current_components = whole_table[0]
# df_current_components.set_index(['Symbol'],inplace=True)
# df_current_components.to_excel("SP500 current components.xlsx") 
cur_components_df = pd.read_excel('SP500 current components.xlsx')
# cur_components_df.set_index(['Symbol'],inplace=True)
cur_components_df.head()

In [ ]:
# Change BF.B and BRK.B to BF-B and BRK-B
for i in cur_components_df.index:
    if '.' in cur_components_df.loc[i,'Symbol']:
        temp_ticker = cur_components_df.loc[i, 'Symbol']
        temp_ticker = temp_ticker.split('.')
        new_ticker = '-'.join(temp_ticker)
        cur_components_df.loc[i,'Symbol'] = new_ticker
        print(new_ticker)
        

In [ ]:
cur_components_df.to_excel("SP500 current components.xlsx")

##### Obtain SP500 changes (from 2015.07 to end of 2022.11)

In [ ]:
# d_changes = whole_table[1][:201]
# d_changes.set_index(['Date'],inplace=True)
# crude_dates = d_changes.index
# df_changes = d_changes.copy()
# new_dates = []
# for i in range(crude_dates.shape[0]):
#     date_ = str(crude_dates[i])
#     date = datetime.strptime(date_, "('%B %d, %Y',)")
#     new_dates.append(date)
# df_changes['Date'] = new_dates
# df_changes.set_index(['Date'],inplace=True)
# df_changes.to_excel("SP500 changes.xlsx")
# Here I then do some manual manipulations so that it's the current version
# I (1) rearrange the first several lines;
# (2) on 2022-06-21, it's 'UA/UAA' -- under armour, they're removed together
# so I insert a row to separate UA and UAA
# (3) on 2017-8-29 'Q'--Quintiles and IMS Health is added, but in current list, it appears as 'IQV'
# So I manually change 'Q' to 'IQV', same as the ticker name at yahoo finance
# (4) on 2016-1-5, Willis Towers Watson is added as 'WLTW', but in current list and yf
# it appears as 'WTW', so I change 'WLTW' to 'WTW'
changes_df = pd.read_excel('SP500 changes.xlsx')
changes_df.set_index(['Date'],inplace=True)
changes_df = changes_df.iloc[:176,:]
changes_df

In [ ]:
import yfinance.shared as shared

In [ ]:
# Test from 2017-01-01 to 2022-10-31, Remove all tickers that unavailable from yahoo finance
start = '2017-01-01'
end = '2022-10-31'

In [ ]:
# find them through removed tickers
delisted_ticker = changes_df.copy()[['Removed Ticker','Removed Security', 'Reason']]
delisted_ticker.dropna(inplace=True)

In [ ]:
# I find these tickers are unavailable
noaccess_ticker = ['CTXS','CERN','PBCT','INFO','XLNX','KSU','MXIM','ALXN','FLIR','CXP','TIF','NBL','ETFC','AGN','RTN']
noaccess_ticker.extend(['XEC','WCG','STI','CELG','TSS','APC','RHT','GGP','DPS','BCR','LVLT','SPLS','WFM','CXO','VIAB','VAR'])
noaccess_ticker.extend(['RAI','YHOO','MJN','LLTC','ENDP','STJ','TYC''CPGX','TE' ,'CVC','ARG', 'TWC','SNDK','TYC','CPGX'])
noaccess_ticker.extend(['CAM','ESV','GMCR','SIAL','ARG','TWC','SNDK','CAM','ESV','GMCR','SIAL','DTV','FDO','LO','ACE','NLSN'])
noaccess_ticker.extend(['FTR', 'DWDP', 'WYN', 'HCBK', 'BRCM', 'LM', 'KRFT', 'HFC', 'BXLT', 'ADS', 'QEP', 'HSP', 'JOY', 'CMCSK'])
# filter out unavailable tickers
delisted = delisted_ticker['Removed Ticker'].values
available = [ticker for ticker in delisted if ticker not in noaccess_ticker]
len(available)

### Before running PCA_graph notebook,  the following code should be run to check stocks availability in Yahoo finance

In [ ]:
# Test if I have any tickers missing
# yf.download(available, start,end)
# fail_tickers = list(shared._ERRORS.keys())
# print(fail_tickers)

In [ ]:
# test = list(cur_components_df.Symbol.values)
# yf.download(test, start,end)
# print(list(shared._ERRORS.keys()))

##### Obtain components mark

In [ ]:
# # generate SP500 components from now to historical periods.
# # initialize
# components_mark_df = pd.DataFrame()
# prev_date = 0
# stocks = list(cur_components_df['Symbol'])
# # generate historical components based on changes
# for date in changes_df.index[:-1]:
#     if date != prev_date:
#         components_mark_df.loc[date, stocks] = 1
#         prev_date = date
#         removed = changes_df.loc[date, 'Removed Ticker']
#         added = changes_df.loc[date, 'Added Ticker']
        
#         if type(removed) == str:
#             stocks.append(removed)            
#         elif isinstance(removed, pd.Series):
#             for ele in list(removed.values):
#                 if type(ele) == str: stocks.append(ele)
                    
#         if type(added) == str:
# #             print(added,date)
#             stocks.remove(added)
#         elif isinstance(added, pd.Series):
#             for ele in list(added.values):
#                 if type(ele) == str: stocks.remove(ele)
                    
# components_mark_df = components_mark_df.sort_index()
# components_mark_df.head()

# save results
# components_mark_df.index.name = 'date'
# components_mark_df.to_csv('components_mark_df.csv')
components_mark_df = pd.read_csv('components_mark_df.csv')
components_mark_df.set_index('date',inplace=True)
components_mark_df

In [ ]:
remove = [t for t in noaccess_ticker if t in components_mark_df.columns.values]
# Remove tickers that not available from Yahoo Finance
components_mark_df.drop(remove, axis=1,inplace = True)
# Change BF.B and BRK.B to BF-B and BRK-B
components_mark_df.rename(columns={'BF.B':'BF-B','BRK.B':'BRK-B'},inplace=True)

In [ ]:
components_mark_df.to_csv('components_mark_df.csv')

##### Components prices new

In [ ]:
all_stocks_list = list(components_mark_df.columns)
startdate = '2015-07-01'
enddate = '2022-11-01'
# data = yf.download(all_stocks_list,startdate,enddate)
# data.to_excel("Data downloaded from now.xlsx")
# manually adjusted the first rows so that column names are right
# reference_data = pd.read_excel('Data downloaded from now.xlsx')

In [ ]:
# reference_data.set_index('Date',inplace=True)
# reference_data

In [ ]:
# adj_close = yf.download(all_stocks_list,startdate,enddate)['Adj Close']
# adj_close.to_excel("prices.xlsx")

In [ ]:
adj_close = pd.read_excel("prices.xlsx")

In [ ]:
adj_close.set_index('Date',inplace=True)
adj_close

In [ ]:
# Remove tickers that not available from Yahoo Finance
adj_close_update = adj_close.copy().drop(remove, axis=1)
# Change BF.B and BRK.B to BF-B and BRK-B
adj_close_update.rename(columns={'BF.B':'BF-B','BRK.B':'BRK-B'},inplace=True)

In [ ]:
adj_close_update.to_excel("prices.xlsx")